In [1]:
# Pytorch model to do classification. Dataset from folder

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import pandas as pd
from PIL import Image
import os

In [21]:
# Use pretrained model (Resnet)
model = torchvision.models.resnet18(pretrained=True)

/home/venom/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/venom/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [22]:
# Input RGB image, normalise to resnet18 and all those stuff
transform = transforms.Compose(
    [transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225])])


In [23]:
# Load dataset from folder
dataset = torchvision.datasets.ImageFolder(root='/home/venom/repo/Stylumia-Internship-Kaggle/Dataset/Modified/', transform=transform)

In [24]:
# Random split dataset into train and test
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
trainset, testset = torch.utils.data.random_split(dataset, [train_size, test_size])


In [25]:
# Dataloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                            shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                            shuffle=False, num_workers=2)


In [26]:
# Freeze all layers except last fc layer
for param in model.parameters():
    param.requires_grad = False
    
# Change last fc layer to 7 classes
model.fc = nn.Linear(512, 7)

# Make sure last layer is trainable
# Make sure the last layer is trainable
for param in model.fc.parameters():
    param.requires_grad = True


In [27]:
# Use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# load model checkpokint
model.load_state_dict(torch.load(
    '/home/venom/repo/Stylumia-Internship-Kaggle/RawData/model.pth'))


<All keys matched successfully>

In [28]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [37]:
# Train, test in each epoch with tqdm progress bar, use functions for train and test
from tqdm import tqdm

def train(trainloader):
    model.train()
    running_loss = 0.0
    for i, data in tqdm(enumerate(trainloader, 0)):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    print("Training loss: ", running_loss/len(trainloader))

def test(testloader):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(testloader, 0)):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    print("Testing loss: ", running_loss/len(testloader))
    print("Accuracy: ", correct/total)


In [30]:
# Train and test for 10 epochs
for epoch in range(10):
    print("Epoch: ", epoch)
    train()
    test()

Epoch:  0


403it [00:32, 12.50it/s]

Training loss:  0.935029268486624



173it [00:14, 11.86it/s]

Testing loss:  0.9105050370872365
Accuracy:  0.6299383834722726
Epoch:  1


...

Testing loss:  0.9235918821627005
Accuracy:  0.6185212033345415
Epoch:  9



403it [00:33, 12.10it/s]

Training loss:  0.9141321833021292



173it [00:14, 12.16it/s]

Testing loss:  0.9009697123759055
Accuracy:  0.6297571583907213


In [31]:
# Unfreeze all layers, and fine tune with smaller learning rate
for param in model.parameters():
    param.requires_grad = True

optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [32]:
# Train and test for 10 epochs
for epoch in range(10):
    print("Epoch: ", epoch)
    train()
    test()


Epoch:  0


403it [00:35, 11.20it/s]

Training loss:  0.8715894214864405



173it [00:14, 11.67it/s]

Testing loss:  0.829567765741679
Accuracy:  0.6652772743747735
Epoch:  1


...

Testing loss:  0.9910242228494214
Accuracy:  0.6678144255164915
Epoch:  9



403it [00:36, 11.16it/s]

Training loss:  0.3072764240482605



173it [00:15, 11.18it/s]

Testing loss:  1.044374104869159
Accuracy:  0.6723450525552737


In [33]:
optimizer = optim.Adam(model.parameters(), lr=0.00001)

In [34]:
# Train and test for 10 epochs
for epoch in range(10):
    print("Epoch: ", epoch)
    train()
    test()


Epoch:  0


403it [00:37, 10.85it/s]

Training loss:  0.21261691645009642



173it [00:15, 11.51it/s]

Testing loss:  1.0509301439530587
Accuracy:  0.676332004349402
Epoch:  1


...

Training loss:  0.1693921140198672



173it [00:15, 11.52it/s]

Testing loss:  1.1346909484766812
Accuracy:  0.6710764769844146
Epoch:  5



260it [00:23, 11.10it/s]


KeyboardInterrupt: 

In [40]:
# train for 5 epochs with test set before submission
trainloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                            shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                            shuffle=False, num_workers=2)
for epoch in range(5):
    print("Epoch: ", epoch)
    train(trainloader)
    test(testloader)


Epoch:  0


173it [00:15, 11.14it/s]

Training loss:  0.9002002665417732



173it [00:15, 11.27it/s]

Testing loss:  0.7938819057335055
Accuracy:  0.7223631750634287
Epoch:  1


...

Testing loss:  0.6771740456881551
Accuracy:  0.7442914099311345
Epoch:  4



173it [00:15, 10.95it/s]

Training loss:  0.733720707238754



173it [00:14, 11.68it/s]

Testing loss:  0.645934400358641
Accuracy:  0.7533526640086988


In [41]:
test(testloader)

173it [00:14, 11.57it/s]

Testing loss:  0.645934400358641
Accuracy:  0.7533526640086988


In [42]:
# Make predictions with this model, on test directory, and epxort to csv as per kaggle format
import pandas as pd
import os
from PIL import Image

# Load test directory
test_dir = '/home/venom/repo/Stylumia-Internship-Kaggle/Dataset/test'

# Submission df structure - file_name,label - Lable is predicted class (0-6)
submission_df = pd.DataFrame(columns=['file_name', 'label'])


# Iterate over all files in test directory
for file in tqdm(os.listdir(test_dir)):
    # Open image, resize, convert to tensor, normalise
    img = Image.open(os.path.join(test_dir, file))
    img = transform(img)
    img = img.unsqueeze(0)
    img = img.to(device)
    
    # Get prediction
    output = model(img)
    _, predicted = torch.max(output.data, 1)
    
    # Add to submission df using pandas concat
    submission_df = pd.concat([submission_df, pd.DataFrame([[file, predicted.item()]], columns=['file_name', 'label'])], ignore_index=True)


100%|██████████| 5751/5751 [00:50<00:00, 114.30it/s]


In [43]:
# Export to csv
submission_df.to_csv('submission_complex_stuff.csv', index=False)

In [44]:
# Save model
torch.save(model.state_dict(), 'complex_stuff.pth')


: 